In [1]:
import pandas as pd

In [2]:
from amplpy import AMPL,Environment
ampl = AMPL(Environment('/home/asino/Downloads/ampl_linux-intel64/'))
ampl.setOption("solver","/home/asino/Downloads/ampl_linux-intel64/cplex")

In [3]:
bought=pd.read_csv("auction.csv")
dictOfMyPlayers={}
dictOfPlayersBelongingToOtherBidders={}
for idx,player in bought.iterrows():
    if player["Owner"]=="me":
        dictOfMyPlayers[player["Name"]]=player["Price"]
    else:
        dictOfPlayersBelongingToOtherBidders[player["Name"]]=player["Price"]


In [4]:
ampl.read("online-auction.mod")
ampl.readData("fantasy-auction-mantra-online.dat")

	online-auction.mod
	line 39 offset 1150
	Caution: 0-dimensional slice


In [ ]:
def suggestTeam(dictOfMyPlayers,dictOfPlayersBelongingToOtherBidders):
    plas=ampl.getSet("Players").getValues()
    valuesForOtherPlayersAmpl={}
    valuesForMyPlayersAmpl={}
    for name in plas:
        name=name[0]
        #print(name)
        valuesForMyPlayersAmpl[name]=0
        valuesForOtherPlayersAmpl[name]=0
    budget=500
    for player,cost in dictOfPlayersBelongingToOtherBidders.items():
        valuesForOtherPlayersAmpl[player]=1
    for player,cost in dictOfMyPlayers.items():
        budget-=cost
        valuesForMyPlayersAmpl[player]=1
        
    if valuesForOtherPlayersAmpl:
        ampl.getParameter("alreadyBoughtByOther").setValues(valuesForOtherPlayersAmpl)
    if valuesForMyPlayersAmpl:
        ampl.getParameter("alreadyBoughtByMe").setValues(valuesForMyPlayersAmpl)
        
        
    
    ampl.getParameter("budget").set(budget)
    ampl.solve()


In [ ]:
suggestTeam(dictOfMyPlayers,dictOfPlayersBelongingToOtherBidders)

In [ ]:
buy=ampl.getVariable("putPlayerInSlot")
cost=ampl.getParameter("cost")
moduleChosen=ampl.getVariable("bidderChoosesModule")
solution=buy.getValues().toDict()
costs=cost.getValues().toDict()
modules=moduleChosen.getValues().toDict()
moduleUsed="AAA"
for moduleName,taken in modules.items():
    if taken>0.5:
        print("Module: {}\n\n".format(moduleName))
        moduleUsed=moduleName

myPls=[]
for key,val in solution.items():
    if key[0]==moduleName:
        if val>0.5:
            myPls.append(key[1])
        

In [ ]:
for player in myPls:
    message="{} {}".format(player,costs[player])
    print(message)